In [1]:
import urllib.request, json 
import json
import numpy as np
#import pycountry as pc
import pandas as pd
import csv
import itertools
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

In [6]:
init_notebook_mode(connected=True)

#EXTRACTING DATA AND CONVERTING IT TO A 1-DIMENSIONAL ARRAY

with urllib.request.urlopen("http://api.population.io:80/1.0/countries") as url:
    slash = "/"
    api_data = json.loads(url.read().decode())
    twoD_data = np.array(list(api_data.values())) #turns the dictionary into a 2D array
    all_countries = twoD_data.ravel()
    country_ready_for_fetch = []
    
#FORMATTING COUNTRIES FOR URL-CALLS LATER
    
country_code_data = pd.read_csv('https://pkgstore.datahub.io/core/country-list/data_csv/data/d7c9d7cfb42cb69f4422dec222dbbaa8/data_csv.csv')
df1 = country_code_data[['Name','Code']]
df2 = country_code_data[['Code']]

country_and_countrycodes = df1.values
countrycodesTwoD = df2.values
countrycodes = countrycodesTwoD.ravel()

chained_country_list = set(itertools.chain.from_iterable(country_and_countrycodes)) & set(all_countries)



for country in chained_country_list:
    try:
        if '/' not in country:
            j = country.replace(" ","%20")
            country_ready_for_fetch.append(j)
                
    except AttributeError:
        pass
    
#EXTRACTING THE POPULATION DATA

date = '/2013-01-01'

country_population = []

for country in country_ready_for_fetch:
    url = "http://api.population.io:80/1.0/population/"

    with urllib.request.urlopen(url + country + date) as url:
        data = json.loads(url.read().decode())
        country_population.append(country)
        country_population.append(data)


result = {}

for i in range(0, len(country_population), 2):
    result[country_population[i]] = country_population[i+1]['total_population']['population']
    
result = {key.replace('%20', ' '): value for key, value in result.items()}



l = np.array(country_and_countrycodes).tolist()

for i in l:
    i.append(result.get(i[0]))

df = pd.DataFrame(l)
df.columns = ['Country','Country-code','Population']


data = dict(type = 'choropleth', 
           locations = df['Country-code'],
           z = df['Population'],
           text = df['Country'],
           colorbar = dict(title = 'Population in thousands'))

layout = dict(title = '2013 global Population', 
             geo = dict(showframe = False,
                       projection = {'type': 'stereographic'}))

choromap3 = go.Figure(data = [data], layout = layout)

#plot(choromap3)

In [7]:
df.head()

,Country,Country-code,Population
0,Afghanistan,AF,30206627.0
1,Åland Islands,AX,NaN
2,Albania,AL,2881160.0
3,Algeria,DZ,37812804.0
4,American Samoa,AS,NaN
